**Domain Background**

What determines a student’s skill level at a math problem? Can I predict whether a student will answer an algebra or geometry problem correctly on the first try? If I know that a student is proficient at one commonly-encountered algebra skill, such as combining like terms or removing coefficients, can I predict what other skills they would be proficient at?

As someone who started out in this field designing and teaching standardized test prep courses and working with individual students to predict their strengths and weaknesses, I developed an ability to intuit what types of skills a student would be good at based on what I already knew they were good at. Being able to group together skills to test based on my domain knowledge of the problem gives me a starting point in identifying clusters of similarity between skills.

**Problem Statement**

Primarily this is a **supervised learning** task, namely a **binary classification** problem, and in the course of working on that problem I expect to do data exploration and some unsupervised learning exploration of potential related categories. The goal of this classification problem is to predict whether the target **Correct First Attempt** will have a value of 1 (meaning the student answered the step correctly on the first try and received no hints) or 0 (the student did not answer the step correctly on the first try or received hints) for each row. This will be discussed further in the next section.

Part of my goal in this project is to find problems that are similar to each other in terms of student success at those problems. Grouping problems by knowledge components (KCs) involved, I want an answer to the question “if a student does well on problems involving KC <x>, what knowledge components will be present in other problems they tend to do well on?”

One way I can define proficiency at a KC is by looking at the number of times a student answers a question containing a particular KC correctly on the first try as a fraction of how many times they encountered questions containing that KC. This is a simplified assessment; student performance tends to improve over time, so a more detailed model would take temporal relationships into account; students only see remedial problems if they don’t do well on the original problems they are given, so a more detailed model would take such causal relationships into account. 

In [1]:
import pandas as pd
import numpy as np
import csv as csv
import matplotlib.pyplot as plt
%matplotlib inline 
import keras
from keras.utils import np_utils
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import make_classification

Using TensorFlow backend.


**Dataset and Inputs**

The data I use is provided by PSLC DataShop in the form of records of interactions between students and computer-aided math tutoring software. The data is structured into four key components: Problem, Step, Knowledge Component (KC), and Opportunity. More detailed information on the data format can be found [here](https://pslcdatashop.web.cmu.edu/KDDCup/rules_data_format.jsp).



For this data exploration section I am using the 2005-2006 algebra training set, which I will be dividing into training, testing, and cross-validation samples during the model-building process. Some relevant information about this dataset is explored below.

In [2]:
df = pd.read_csv('algebra_2005_2006_train.txt', sep='\t')

print ("This dataset contains {0} rows.".format(df.shape[0]))
print ("This dataset contains {0} columns.".format(df.shape[1]))
print ("This dataset contains {0} unique student IDs.".format(len(df['Anon Student Id'].unique())))

This dataset contains 809694 rows.
This dataset contains 19 columns.
This dataset contains 574 unique student IDs.


Each of the 809694 rows in this dataset corresponds to a **"student-step."** Each problem a student is asked to solve is divided into predetermined steps similar to "Eliminate the parentheses in this expression by distributing x" or "Remove the coefficient from y in this equation." Each step is part of a more complex task, such as, for a geometry problem, finding the surface area of a three-dimensional object.  

Each of the 574 individual student in this dataset is identified by a unique string **Anon Student Id**. The first five rows of the dataframe are shown below.

In [3]:
df.head(5)

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
0,1,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG4-FIXED,1,3(x+2) = 15,2005-09-09 12:24:35.0,2005-09-09 12:24:49.0,2005-09-09 12:25:15.0,2005-09-09 12:25:15.0,40.0,NaN,40.0,0,2,3,1,[SkillRule: Eliminate Parens; {CLT nested; CLT...,1
1,2,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG4-FIXED,1,x+2 = 5,2005-09-09 12:25:15.0,2005-09-09 12:25:31.0,2005-09-09 12:25:31.0,2005-09-09 12:25:31.0,16.0,16.0,NaN,1,0,0,1,"[SkillRule: Remove constant; {ax+b=c, positive...",1~~1
2,3,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,1,2-8y = -4,2005-09-09 12:25:36.0,2005-09-09 12:25:43.0,2005-09-09 12:26:12.0,2005-09-09 12:26:12.0,36.0,NaN,36.0,0,2,3,1,"[SkillRule: Remove constant; {ax+b=c, positive...",2
3,4,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,1,-8y = -6,2005-09-09 12:26:12.0,2005-09-09 12:26:34.0,2005-09-09 12:26:34.0,2005-09-09 12:26:34.0,22.0,22.0,NaN,1,0,0,1,"[SkillRule: Remove coefficient; {ax+b=c, divid...",1~~1
4,5,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,2,-7y-5 = -4,2005-09-09 12:26:38.0,2005-09-09 12:28:36.0,2005-09-09 12:28:36.0,2005-09-09 12:28:36.0,118.0,118.0,NaN,1,0,0,1,"[SkillRule: Remove constant; {ax+b=c, positive...",3~~1


The target column is **Correct First Attempt** and it will take a value of 1 if the student answered the problem correctly on the first try without asking for a hint from the program, and 0 if they did not.

The primary feature I will focus on will be the **KC(Default)** column (and potentially the **Opportunity(Default)** column in a temporal analysis). In the test dataset for this competition, many of the timing and result-specific columns are not provided, such as **Step Start Time**, **Step Duration**, **Incorrects**, and **Hints**. I will therefore not be using those columns in the classifiers I will be building, because while they easily can be used to predict the target column, they provide less interesting information than the features related to the educational subject areas.

**Solution Statement**

The feature I will primarily focus on in this data exploration and solution will be **KC(Default)**. KCs (knowledge components) are determined by researchers and assigned to each student-step (row). They identify the skills used in solving a step correctly. A full list of KCs in this dataset will be examined below. 

Each row (student-step) contains several KCs. The column **KC(Default)** contains a string of these KC strings separated by tilde (~) characters. KCs can be treated as a categorical variable for this problem. I have split the **KC(Default)** column into individual unique KC strings, one-hot encoded the categorical KCs, and pickled the resulting dataframe (full process not shown).

In [3]:
df_mt = pd.read_pickle('mtpickle.p')
print ("This dataset contains {0} knowledge component features.".format(len(df_mt.columns[18:])))

NameError: name 'pd' is not defined

In [4]:
df_mt.head(5)

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,factor-sp,perform-mult-r-sp,perform-mult-row2-sp,perform-mult-sp,perform-mult-whole-sp,qft-den-sp,qft-num1-sp,qft-num2-sp,simplify-fractions-sp,KC(Default)
0,1,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG4-FIXED,1,3(x+2) = 15,2005-09-09 12:24:35.0,2005-09-09 12:24:49.0,2005-09-09 12:25:15.0,2005-09-09 12:25:15.0,...,0,0,0,0,0,0,0,0,0,[[SkillRule: Eliminate Parens; {CLT nested; CL...
1,2,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG4-FIXED,1,x+2 = 5,2005-09-09 12:25:15.0,2005-09-09 12:25:31.0,2005-09-09 12:25:31.0,2005-09-09 12:25:31.0,...,0,0,0,0,0,0,0,0,0,"[[SkillRule: Remove constant; {ax+b=c, positiv..."
2,3,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,1,2-8y = -4,2005-09-09 12:25:36.0,2005-09-09 12:25:43.0,2005-09-09 12:26:12.0,2005-09-09 12:26:12.0,...,0,0,0,0,0,0,0,0,0,"[[SkillRule: Remove constant; {ax+b=c, positiv..."
3,4,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,1,-8y = -6,2005-09-09 12:26:12.0,2005-09-09 12:26:34.0,2005-09-09 12:26:34.0,2005-09-09 12:26:34.0,...,0,0,0,0,0,0,0,0,0,"[[SkillRule: Remove coefficient; {ax+b=c, divi..."
4,5,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,2,-7y-5 = -4,2005-09-09 12:26:38.0,2005-09-09 12:28:36.0,2005-09-09 12:28:36.0,2005-09-09 12:28:36.0,...,0,0,0,0,0,0,0,0,0,"[[SkillRule: Remove constant; {ax+b=c, positiv..."


A full list of all 112 unique KCs is provided below. I have sorted it in order of the number of rows (student-steps) each KC appears in. The most common KC, **"Entering a given,"** appears in 76894 different student-steps, or about 9.5% of the full dataset, while the least common KC, **[SkillRule: Do Multiply - Whole nested; [Typei...,** appears in only one student-step. 

To cut down on the number of features, I will be dropping the columns for "outlier" KCs that appear in fewer than some predetermined cutoff percentage of student-steps, such as 0.25%. I will also be focusing further data exploration more heavily on the most common KCs (appearing in >20000 student-steps) to determine which of them tend to appear together. 

In [10]:
df_count.to_pickle('kclistpickle.p')

In [2]:
df_kc = df_mt[df_mt.columns[18:-1]]
df_count = df_kc.apply(lambda x: x.value_counts())
df_count = df_count[1:].melt()
df_count = df_count.sort_values(by=['value'], ascending=False)

NameError: name 'df_mt' is not defined

In [1]:
df_kc

NameError: name 'df_kc' is not defined

In [106]:
pd.crosstab(df_kc['Find Y, any form'],df_kc['Using large numbers'],margins=True)

Using large numbers,0,1,All
"Find Y, any form",,,
0,780963,26554,807517
1,1847,330,2177
All,782810,26884,809694


Using crosstabs will be one potential method for me to identify overlap between KCs. As indicated above, **"Using large numbers"** and **"Find Y, any form"** appear together in 330 rows. 

These are some of the additional questions I plan on including in my analysis: 

- Average number of KCs in each row
- Groups of KCs that appear together
- Maximum percentage of the dataset can be covered with minimum percentage of the number of KCs

I am planning to use a strong classification algorithm such as random forests for the prediction section of the analysis. I have also constructed a sequential neural network that I plan on including in order to compare its performance and results. 

**Benchmark Model**

I am using a dummy classifier as a benchmark model. The first dataset I use will include only the KCs as features and does not include any information about the problem name, section of the curriculum, or any identifying information about the student such as the student ID. It also does not include any information that would not have been included in the final test set portion of the KDD Challenge, such as the problem times or hints and incorrects. 

In [108]:
df_mt = df_mt.drop('Step Start Time', 1)
df_mt = df_mt.drop('First Transaction Time', 1)
df_mt = df_mt.drop('Correct Transaction Time', 1)
df_mt = df_mt.drop('Step End Time', 1)
df_mt = df_mt.drop('Step Duration (sec)', 1)
df_mt = df_mt.drop('Correct Step Duration (sec)', 1)
df_mt = df_mt.drop('Error Step Duration (sec)', 1)
df_mt = df_mt.drop('Incorrects', 1)
df_mt = df_mt.drop('Hints', 1)
df_mt = df_mt.drop('Corrects', 1)
df_mt = df_mt.drop('Opportunity(Default)', 1)
df_mt = df_mt.drop('Problem Hierarchy', 1)
df_mt = df_mt.drop('Problem View', 1)
df_mt = df_mt.drop('Step Name', 1)
df_mt = df_mt.drop('Anon Student Id', 1)
df_mt = df_mt.drop('Row', 1)
df_mt = df_mt.drop('Problem Name', 1)

In [110]:
df_mt['is_train'] = np.random.uniform(0, 1, len(df_mt)) <= .75
train, test = df_mt[df_mt['is_train']==True], df_mt[df_mt['is_train']==False]
df_mt = df_mt.drop('Correct First Attempt',1).join(df_mt['Correct First Attempt']) #make CFA the last column

# Show the number of observations for the test and training dataframes
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:', len(test))

Number of observations in the training data: 607530
Number of observations in the test data: 202164


In [113]:
features = df_mt.columns[:-1]
target = df_mt.columns[-1]

In [116]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

clf = DummyClassifier(strategy='most_frequent')
clf.fit(train[features], train[target])
y_pred = clf.predict(test[features])

accuracy_score(y_pred, test[target])

0.76708513879820339

In [117]:
df_mt['Correct First Attempt'].value_counts()

1    620642
0    189052
Name: Correct First Attempt, dtype: int64

Because this data is unbalanced, (about 77% of the observations have target “Correct First Attempt” values of 1 and 23% have target values of 0), this model can achieve an accuracy of 77% by simply guessing 1 for every observation (the "most_frequent" strategy). This is not an adequate solution because we also want to be able to identify negative cases accurately, i.e. students who do not answer questions correctly on the first try. This 77% “success” rate would be due to the classes being unbalanced, not to the classifier being useful. 

I am not necessarily expecting this to be the only set of features or final benchmark model I will use, since I am planning to integrate other features into future models besides just the KCs. This is simply what I am starting with because it demonstrates that the data is unbalanced and that this is the minimum level of overall accuracy I could expect from an overly simplified method. 

**Evaluation Metrics**

As expected, using a dummy classifier with strategy “most_frequent” (predicting that all observations will have the most frequent value) gives an accuracy of 76.7%. The dummy classifier guesses 1 for every row. Therefore I have 100% accuracy for rows with CFA values of 1 and 0% accuracy for rows with CFA values of 0. 

In [127]:
np.unique(y_pred) # verify that the classifier has guessed 1 for every row 

array([1])

Based on this, I want to focus on maximizing accuracy in identifying the minority class. Incidentally I now also know that any classifier that achieves an overall accuracy of less than 77% could be outperformed by a dummy classifier that guessed 1 for every observation. Overall, metrics like accuracy will probably not be the most useful in this dataset because the classes are unbalanced. I would use a method such as F1 score (combining precision and recall).

**Project Design**

The data I are using is provided by PSLC DataShop in the form of records of interactions between students and computer-aided math tutoring software. Each individual record is referred to as a student-step and indicates whether a student answered a particular step in a math problem correctly the first time without receiving any hints. 

I will begin the report by presenting descriptive statistics on the dataset such as the number of students, the average number of questions each student answers correctly on the first try, etc., for the purposes of exploring the dataset and presenting a clear picture of how to interpret the results of the later analysis. 

One useful source of information is likely to be the Knowledge Components (KCs) and the corresponding Opportunity Counts (for temporal analysis). Whether or not there is a correlation strong enough to provide significant predictive power, the combination of KCs present in a step would be a useful piece of information to be able to connect with the number of Correct First Attempts a student gets. I want to be able to see what relationships there are between different groups of KCs and student performance. 

Because this is a binary categorical problem I are solving, in deciding which columns to use as features for predictive modeling, I want to discover axes along which the data is linearly separable. I will need to process features such as KC which are given to us as strings into categories that I can use through methods such as one-hot encoding. 

The predictive models I will use will be some combination of supervised learning models such as random forests. Part of my goal will also be to identify similarity clusters of skillsets and similarities between neighboring student-step points using unsupervised clustering methods and PCA dimension reduction. I have also  included some visualizations and code snippets in my outline above to illustrate some of the questions I will be focusing on.